In [15]:
import requests
import urllib
import csv
from bs4 import BeautifulSoup

In [2]:
def get_topic_list_html():
    r = requests.get('http://spectrum.ieee.org/static/topics')
    
    soup = BeautifulSoup(r.text, "lxml")
    return soup

In [3]:
def get_topic_list(html):
    tmp_list = []
    
    tmp = html.find_all(class_="article-detail")[1]
    tmp_ul = tmp.find_all('ul')[0]
    tmp_li = tmp_ul.find_all('a')
    
    for i in range(1,len(tmp_li)):
        tmp_str = str(tmp_li[i]).split("\"")[1]
        if(tmp_str.count('/') == 2):
            tmp_list.append(tmp_str)
    
    return tmp_list

In [4]:
def get_article(topic_link):
    
    tmp_list = []
    
    r = requests.get('http://spectrum.ieee.org' + topic_link)
    tmp_html = BeautifulSoup(r.text, "lxml")    
    tmp_html = tmp_html.find_all(role="article")[0]
    tmp_html = tmp_html.find_all('a')
    for i in range(0,len(tmp_html)):
        tmp = str(tmp_html[i]).split("\"")[1]
        tmp_list.append(tmp)
        
    return tmp_list
    #print tmp

In [5]:
def get_article_detail(link):
    r = requests.get('http://spectrum.ieee.org'+link)    
    tmp_html = BeautifulSoup(r.text, "lxml")
    
    try:
        tmp_html = tmp_html.find_all(id="medium-content")[0]
        tmp_title = tmp_html.find_all(class_="article-title")[0].get_text()
        tmp_date = tmp_html.find_all(role="contentinfo")[0].get_text().split("\n")[4]
        tmp_content = tmp_html.find_all(class_="entry-content")[0].get_text()
        article = {'title': tmp_title, 'date': tmp_date, 'content': tmp_content}
    except:
        article = {'title': "None", 'date': "None", 'content': "None"}
        return article
    return article

In [6]:
test_html = get_topic_list_html()
topic_list = get_topic_list(test_html)

In [7]:
article = []

In [8]:
for link in topic_list:
    tmp_list = get_article(link)
    for art_link in tmp_list:
        article.append(get_article_detail(art_link))

ConnectionError: HTTPConnectionPool(host='spectrum.ieee.orghttp', port=80): Max retries exceeded with url: //www.flightglobal.com/articles/2008/01/03/220564/faa-demands-connectivity-security-for-boeing-787-control-and-information.html (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0000000005D1D748>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [10]:
print len(article)

2016


In [16]:
out_path= "outfile.txt"
out_file = open(out_path, 'wb')

fieldnames = sorted(list(set(k for d in article for k in d)))
writer = csv.DictWriter(out_file, fieldnames=fieldnames, dialect='excel')

writer.writeheader() # Assumes Python >= 2.7
for row in article:
    writer.writerow(row)
out_file.close()

UnicodeEncodeError: 'ascii' codec can't encode character u'\xa0' in position 35: ordinal not in range(128)